# Setup

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from random import randint
import math
from datetime import datetime
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\DecisionTree'

In [3]:
for i in range(3):
    os.chdir("..")

In [4]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification'

In [5]:
main_data_dir = os.getcwd() + "\\Data set"
kmean_data_dir = main_data_dir + "\\kmean_dataset"
kmean_data_10cv_dir = kmean_data_dir + "\\10cv_128"
import glob

kmean_data_10cv_lst = [x for x in glob.glob(kmean_data_10cv_dir + '\\*') if 'surf_test_0' in x or 'surf_train_0' in x and 'index' in x]
print(kmean_data_10cv_lst)

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_test_0_4_3_True_Falsehisto_128_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\sur

In [6]:
kmean_train_paths = [x for x in kmean_data_10cv_lst if 'train' in x]
kmean_test_paths = [x for x in kmean_data_10cv_lst if 'test' in x]
print(kmean_train_paths)
print()
print(kmean_test_paths)
print(len(kmean_train_paths))
print(len(kmean_test_paths))

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_128\\surf_train_0_4_3_True_Falsehisto_128_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_1

In [7]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """to_categorical _summary_

    Arguments:
        y -- The label of the data set with the shape of [None, 1]

    Keyword Arguments:
        num_classes -- The num_classes in the data set (default: {None})
        dtype -- the type of each element of the label after reshape (default: {'float32'})

    Returns:
        the label of the data set with the shape of [number of samples, number of classes].
    """    
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [8]:
class roc_curve_score:
    def __init__(self, y_true, y_pred, num_class):
        self.y_test = to_categorical(y_true)
        self.y_score = to_categorical(y_pred)
        print(np.unique(self.y_test), np.unique(self.y_score))
        self.fprs = {}
        self.tprs = {}
        self.thresh_holds = {}
        for x in range(num_class):
            self.fprs[x], self.tprs[x], self.thresh_holds[x] = roc_curve(self.y_test[:, x], self.y_score[:, x], drop_intermediate=False)
            self.fprs[x] = self.fprs[x].tolist()
            self.tprs[x] = self.tprs[x].tolist()
            self.thresh_holds[x] = self.thresh_holds[x].tolist()
        
        self.fpr_micro_avg, self.tpr_micro_avg, self.threshold_micro_avg, = roc_curve(self.y_test.ravel(), self.y_score.ravel())
        
        all_fpr = np.unique(np.concatenate([self.fprs[i] for i in range(num_class)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(num_class):
            mean_tpr += np.interp(all_fpr, self.fprs[i], self.tprs[i])
        mean_tpr /= num_class
        self.fpr_macro_avg = all_fpr.tolist()
        self.tpr_macro_avg = mean_tpr.tolist()
    
    def get_tpr(self, _class = None):
        if(_class):
            return self.tprs[_class]
        else:
            return self.tprs
    
    def get_fpr(self, _class = None):
        if(_class):
            return self.fprs[_class]
        else:
            return self.fprs

    def get_thresholds(self, _class = None):
        if(_class):
            return self.thresh_holds[_class]
        else:
            return self.thresh_holds

    def get_roc_dict(self):
        return {
            "tpr" : self.get_tpr(),
            "fpr" : self.get_fpr(),
            "thresholds" : self.get_thresholds(),
            "fpr_micro_avg" : self.fpr_micro_avg.tolist(),
            "tpr_micro_avg" : self.tpr_micro_avg.tolist(),
            "fpr_macro_avg" : self.fpr_macro_avg,
            "tpr_macro_avg" : self.tpr_macro_avg,
        }

In [9]:
class utils:
    def __init__(self, confusion_matrix, y_true, y_pred):
        """
            - confusion_matrix: 2x2 numpy array
            - y_true: array of label
            - y_pred: array of output value calculated by model
            - fold_count: number of folds
        """
        
        # Initilize all indicator
        self.TP = confusion_matrix[0][0] # true positive
        self.FN = confusion_matrix[1][0] # false negative
        self.FP = confusion_matrix[1][0] # false positive
        self.TN = confusion_matrix[1][1] # true negative
        self.precision = self.TP/(self.FN +  self.TP) # Precision Score - Positive Predictive Value
        self.recall = self.TP / (self.TP + self.FN)
        self.f1_score =  (2 * self.TP) / (2 * self.TP + self.FP + self.FN)
        self.sensitivity = self.TP / (self.TP + self.FN) # True Positive Rate
        self.specificity = self.TN / (self.TN + self.FP) # False Positive Rate
        self.negative_predictive_value = self.TN / (self.TN + self.FN) # Negative Predictive Value
        self.false_negative_rate = self.FN / (self.FN + self.TP) # False Negative Rate
        self.false_positive_rate = self.FP / (self.FP + self.TN) # False Positive Rate
        self.false_discovery_rate = self.FP / (self.FP + self.TP) # False Discovery Rate
        self.false_omission_rate = self.FN / (self.FN + self.TN) # False Ommision Rate
        self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
        self.negative_likelihood_ratio = self.false_negative_rate / self.specificity # Negative Likelihood Ratio
        self.prevalence_threshold = math.sqrt(self.false_positive_rate) / (math.sqrt(self.sensitivity) + 
                                                                      math.sqrt(self.false_positive_rate)) # Prevalance Threshold
        self.threat_score = self.TP / (self.TN + self.FN + self.FP) # Threat Score
        self.prevalence = (self.TP + self.FN)/(self.TP + self.FN + self.TN + self.FP) # Prevalance 
        #  Matthews correlation coefficient
        self.matthews_correlation_coefficient = (self.TP*self.TN - self.FN*self.FP) / ((self.TP + self.FP)
                                                                                       *(self.TP + self.FN)
                                                                                       *(self.TN + self.FP)
                                                                                       *(self.TN + self.FN))
        self.fowlkes_mallows_index = math.sqrt(self.sensitivity + self.precision) # Fowlkes–Mallows index
        self.informedness = self.sensitivity + self.specificity - 1 # informedness
        self.markedness = self.precision + self.negative_predictive_value - 1 # markedness
        self.diagnostic_odds_ratio = self.positive_likelihood_ratio / self.negative_likelihood_ratio # Diagnostic odds ratio
        self.accuracy = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        self.balanced_accuracy = (self.sensitivity + self.specificity) / 2
        self.roc_auc_macro = roc_auc_score(y_true, y_pred)
        self.roc_auc_micro = roc_auc_score(y_true, y_pred, average = 'micro')
        self.roc_auc_weighted = roc_auc_score(y_true, y_pred, average = 'weighted')
        self.cls_report = classification_report(y_true, y_pred)
        
        # Initilize the structure of output_dicts
        self.confusion_matrix = {
            "TP" : self.TP,
            "TN" : self.TN,
            "FN" : self.FN,
            "FP" : self.FP,
            "precision" : self.precision,
            "recall" : self.recall,
            "f1_score" : self.f1_score,
            "sensitivity" : self.sensitivity,
            "specificity" : self.specificity,
            "negative_predictive_value" : self.negative_predictive_value,
            "false_negative_rate" : self.false_negative_rate,
            "false_positive_rate" : self.false_positive_rate,
            "false_discovery_rate" : self.false_discovery_rate,
            "false_omission_rate" : self.false_omission_rate,
            "Positive_likelihood_ratio" : self.positive_likelihood_ratio,
            "Negative_likelihood_ratio" : self.negative_likelihood_ratio,
            "prevalence_threshold" : self.prevalence_threshold,
            "threat_score" : self.threat_score,
            "Prevalence" : self.prevalence,
            "Matthews_correlation_coefficient" : self.matthews_correlation_coefficient,
            "Fowlkes_Mallows_index" : self.fowlkes_mallows_index,
            "informedness" : self.informedness,
            "markedness" : self.markedness,
            "Diagnostic_odds_ratio" : self.diagnostic_odds_ratio,
            "accuracy" : self.accuracy,
            "balanced_accuracy" : self.balanced_accuracy
        }
        
        self.roc_auc_score = {
            "Macro": self.roc_auc_macro, 
            "Micro": self.roc_auc_micro,
            "Weight": self.roc_auc_weighted
        }
        
#         self.roc_curve = roc_curve_score(y_true=y_true, y_pred=y_pred, num_class=2).get_roc_dict()
        
        self.sub_dict = {
            "Confusion Matrix" : self.confusion_matrix,
            "ROC_AUC_SCORE" : self.roc_auc_score,
            "Classification Report" : classification_report(y_true.tolist(), y_pred.tolist(), 
                                                            labels = [0, 1], # 0 : Licit, 1 : Illicit
                                                            output_dict = True),
#             "ROC_DRAW" : self.roc_curve
        }
    def get_value(self):
        return self.sub_dict
        
# Test 
y_true = np.array([randint(0,1) for x in range(200)])
y_pred = np.array([randint(0,1) for x in range(200)])
confusion_matrix_test = confusion_matrix(y_true, y_pred)
base_utils = utils(confusion_matrix_test, y_true, y_pred)
base_utils.get_value()

{'Confusion Matrix': {'TP': 48,
  'TN': 48,
  'FN': 46,
  'FP': 46,
  'precision': 0.5106382978723404,
  'recall': 0.5106382978723404,
  'f1_score': 0.5106382978723404,
  'sensitivity': 0.5106382978723404,
  'specificity': 0.5106382978723404,
  'negative_predictive_value': 0.5106382978723404,
  'false_negative_rate': 0.48936170212765956,
  'false_positive_rate': 0.48936170212765956,
  'false_discovery_rate': 0.48936170212765956,
  'false_omission_rate': 0.48936170212765956,
  'Positive_likelihood_ratio': 1.0434782608695652,
  'Negative_likelihood_ratio': 0.9583333333333334,
  'prevalence_threshold': 0.4946802489414612,
  'threat_score': 0.34285714285714286,
  'Prevalence': 0.5,
  'Matthews_correlation_coefficient': 2.407944289800911e-06,
  'Fowlkes_Mallows_index': 1.0105823052798226,
  'informedness': 0.02127659574468077,
  'markedness': 0.02127659574468077,
  'Diagnostic_odds_ratio': 1.0888468809073724,
  'accuracy': 0.5106382978723404,
  'balanced_accuracy': 0.5106382978723404},
 'RO

In [10]:
class Training:
    def __init__(self, fold_count, X, y):
        self.fold_count = fold_count
        self.param_grid = {
            "criterion" : ["gini", "entropy", "log_loss"],
            "splitter" : ["best", "random"],
            "max_features" : ["sqrt", "log2"],
            "class_weight" : ["balanced", None]
        }
        self.kf = KFold(n_splits=fold_count)
        self.history = {}
        self.X = X
        self.y = y
    
    def get_fold_value(self):
        return self.kf        
        
    def training(self, criterion,
                 splitter, 
                 max_features, 
                 class_weight, train_case):
    
        # output_dict initilize
        output_dict = {}
    
        # poiter track the index of fold
        fold_index = 0
    
        for train_index, test_index in zip(self.X, self.y):
            train_df = pd.read_csv(train_index)
            test_df = pd.read_csv(test_index)
            
            print("\tFold: {}".format(fold_index))
            print("\tTRAIN:", train_index, "\n\tTEST:", test_index)
        
            # folding data
            x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
            x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
    
            # Training
            print("\t\tTraining : {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
            model = DecisionTreeClassifier(criterion = criterion, 
                                              splitter = splitter, 
                                              max_features = max_features, 
                                              class_weight = class_weight)
            model.fit(x_train,y_train)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Testing
            print("\t\tValidation: {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
            y_pred = model.predict(x_test)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Evaluation
            cm = confusion_matrix(y_test, y_pred)
            current_utils = utils(cm, y_test, y_pred)
            output_dict["fold_{}".format(fold_index)] = current_utils.get_value()
            
            fold_index += 1
        print("\n")
        return output_dict 
    
    def train(self, path):
        count = 0
        for x in self.param_grid["criterion"]:
            for i in self.param_grid["splitter"]:
                for j in self.param_grid["max_features"]:
                    for k in self.param_grid["class_weight"]:
                        print("Traning Case: {}".format(count))
                        self.history["train_{}".format(count)] = {
                            "param" : {
                            "criterion" : x,
                            "splitter" : i,
                            "max_features" : j,
                            "class_weight" : k
                            },
                            "train_fold" : self.training(x, i, j, k, count)
                        }
                        count += 1
        with open(path.format(self.fold_count), 'wb') as f:
            pickle.dump(self.history, f)
        
        return self.history

# Training

In [11]:
training = Training(10, kmean_train_paths, kmean_test_paths)
approach_dir = os.getcwd() + "\\Experiment\\Approach"
svm_dir = approach_dir + "\\DecisionTree"
save_result_path = svm_dir + '\\rdf_km128_surf0_result_fold{0}_0.9.pkl'
training.train(path = save_result_path)

Traning Case: 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:26:15 --- End: 11/29/2022, 09:26:15
		Validation: 0 -- Start: 11/29/2022, 09:26:15---End: 11/29/2022, 09:26:15
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:26:15 --- End: 11/29/2022, 09:26:15
		Validation: 1 -- Start: 11/29/2022, 09:26:15---End: 11/29/2022, 09:26:15
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv 
	

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index2.csv
		Training : 2 -- Start: 11/29/2022, 09:26:16 --- End: 11/29/2022, 09:26:16
		Validation: 2 -- Start: 11/29/2022, 09:26:16---End: 11/29/2022, 09:26:16
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index3.csv
		Training : 3 -- Start: 11/29/2022, 09:26:16 --- End: 11/29/2022, 09:26:16
		Validation: 3 -- Start: 11/29/2022, 09:26:16---End: 11/29/2022, 09:26:16
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index4.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/29/2022, 09:26:16
		Validation: 7 -- Start: 11/29/2022, 09:26:16---End: 11/29/2022, 09:26:16
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index8.csv
		Training : 8 -- Start: 11/29/2022, 09:26:16 --- End: 11/29/2022, 09:26:16
		Validation: 8 -- Start: 11/29/2022, 09:26:16---End: 11/29/2022, 09:26:16
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index9.csv
		Training : 9 -- Start: 11/29/2022, 09:26:16 --- End: 11/29/2022, 09:26:16
		Validation: 9 -- Start: 11/29/2022, 09:26:16---End: 11/29/2022, 09:26:16


Traning Case: 4
	Fold: 0
	TRAIN: D:\Github\Stomach-Stat

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index2.csv
		Training : 2 -- Start: 11/29/2022, 09:26:21 --- End: 11/29/2022, 09:26:21
		Validation: 2 -- Start: 11/29/2022, 09:26:21---End: 11/29/2022, 09:26:21
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index3.csv
		Training : 3 -- Start: 11/29/2022, 09:26:21 --- End: 11/29/2022, 09:26:21
		Validation: 3 -- Start: 11/29/2022, 09:26:21---End: 11/29/2022, 09:26:21
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index4.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:26:21 --- End: 11/29/2022, 09:26:21
		Validation: 0 -- Start: 11/29/2022, 09:26:21---End: 11/29/2022, 09:26:21
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:26:21 --- End: 11/29/2022, 09:26:21
		Validation: 1 -- Start: 11/29/2022, 09:26:21---End: 11/29/2022, 09:26:21
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv 
	TEST: D:\Github\

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 3,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.6,
     'recall': 0.6,
     'f1_score': 0.6,
     'sensitivity': 0.6,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.4,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.4,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 3.6,
     'Negative_likelihood_ratio': 0.48,
     'prevalence_threshold': 0.3451409985003952,
     'threat_score': 0.21428571428571427,
     'Prevalence': 0.29411764705882354,
     'Matthews_correlation_coefficient': 0.007222222222222222,
     'Fowlkes_Mallows_index': 1.0954451150103321,
     'informedness': 0.43333333333333335,
     'markedness': 0.43333333333333335,
     'Diagnostic_od

# Result Analyzing

In [12]:
file = open(save_result_path.format(10), 'rb')
result = pickle.load(file)
result

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 3,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.6,
     'recall': 0.6,
     'f1_score': 0.6,
     'sensitivity': 0.6,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.4,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.4,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 3.6,
     'Negative_likelihood_ratio': 0.48,
     'prevalence_threshold': 0.3451409985003952,
     'threat_score': 0.21428571428571427,
     'Prevalence': 0.29411764705882354,
     'Matthews_correlation_coefficient': 0.007222222222222222,
     'Fowlkes_Mallows_index': 1.0954451150103321,
     'informedness': 0.43333333333333335,
     'markedness': 0.43333333333333335,
     'Diagnostic_od

In [13]:
print("Number of Training: {}".format(len(list(result.keys()))))

Number of Training: 24


In [14]:
training_case_lst = list(result.keys())
print(training_case_lst[0])

train_0


In [15]:
train_sample = result[training_case_lst[0]]
print(train_sample['param'])
print("Number of Fold: {}".format(len(train_sample['train_fold'])))

{'criterion': 'gini', 'splitter': 'best', 'max_features': 'sqrt', 'class_weight': 'balanced'}
Number of Fold: 10


In [16]:
fold_sample = train_sample['train_fold']['fold_0']
print(fold_sample['Classification Report'])

{'0': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 6}, '1': {'precision': 0.7692307692307693, 'recall': 0.8333333333333334, 'f1-score': 0.8, 'support': 12}, 'accuracy': 0.7222222222222222, 'macro avg': {'precision': 0.6846153846153846, 'recall': 0.6666666666666667, 'f1-score': 0.6727272727272727, 'support': 18}, 'weighted avg': {'precision': 0.7128205128205128, 'recall': 0.7222222222222222, 'f1-score': 0.7151515151515153, 'support': 18}}


In [17]:
result_analyze_dict = {
    'Train' : [],
    'MACC' : [],
    'MAP' : [],
    'MASens' : [],
    'MASpec' : [],
    'MAF1' : [],
    'AUC' : []
}

for result_key in result:
    result_analyze_dict['Train'].append(result_key)
    train_base = result[result_key]
    fold_base = train_base['train_fold']
    acc_lst, pre_lst, re_lst, spec_lst, f1_lst, auc_lst = [], [], [], [], [], []
    for fold_key in fold_base:
        current_fold = fold_base[fold_key]
        current_fold_cls = current_fold['Classification Report']
        acc_lst.append(current_fold_cls['accuracy'])
        pre_lst.append(current_fold_cls['macro avg']['precision'])
        re_lst.append(current_fold_cls['macro avg']['recall'])
        spec_lst.append(current_fold['Confusion Matrix']['specificity'])
        auc_lst.append(current_fold['ROC_AUC_SCORE']['Macro'])
        f1_lst.append(current_fold_cls['macro avg']['f1-score'])
    result_analyze_dict['MACC'].append(sum(acc_lst)/len(acc_lst))
    result_analyze_dict['MAP'].append(sum(pre_lst)/len(pre_lst))
    result_analyze_dict['MASens'].append(sum(re_lst)/len(re_lst))
    result_analyze_dict['MASpec'].append(sum(spec_lst)/len(spec_lst))
    result_analyze_dict['MAF1'].append(sum(f1_lst)/len(f1_lst))
    result_analyze_dict['AUC'].append(sum(auc_lst)/len(auc_lst))

result_df = pd.DataFrame(result_analyze_dict)
result_df.head()

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
0,train_0,0.761111,0.729247,0.716667,0.850000,0.711333,0.716667
1,train_1,0.776923,0.725859,0.724621,0.832576,0.717209,0.724621
2,train_2,0.723504,0.656352,0.662121,0.824242,0.653054,0.662121
3,train_3,0.758974,0.749015,0.704167,0.841667,0.712486,0.704167
4,train_4,0.697863,0.656320,0.633333,0.800000,0.632022,0.633333


In [18]:
result_max = result_df.max()
result_max

Train      train_9
MACC      0.810256
MAP       0.791983
MASens    0.769697
MASpec    0.881818
MAF1      0.764716
AUC       0.769697
dtype: object

In [19]:
id_result_max = result_df.iloc[:, 1:].idxmax()
id_result_max

MACC      19
MAP       19
MASens    18
MASpec    19
MAF1      19
AUC       18
dtype: int64

In [20]:
id_max = 'train_{}'.format(id_result_max.mode()[0])
print(id_max)

train_19


In [21]:
result_df.loc[result_df['Train'] == id_max]

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
19,train_19,0.810256,0.791983,0.765909,0.881818,0.764716,0.765909


In [22]:
train_max = result[id_max]
train_max['param']

{'criterion': 'log_loss',
 'splitter': 'best',
 'max_features': 'log2',
 'class_weight': None}

In [23]:
train_max_fold = train_max['train_fold']
train_max_analyze = {
    'Fold' : [],
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for fold_key in train_max_fold:
    train_max_analyze['Fold'].append(fold_key)
    current_fold = train_max_fold[fold_key]
    current_fold_cls = current_fold['Classification Report']
    train_max_analyze['ACC'].append(current_fold_cls['accuracy'])
    train_max_analyze['AP'].append(current_fold_cls['macro avg']['precision'])
    train_max_analyze['ASens'].append(current_fold_cls['macro avg']['recall'])
    train_max_analyze['ASpec'].append(current_fold['Confusion Matrix']['specificity'])
    train_max_analyze['AF1'].append(current_fold_cls['macro avg']['f1-score'])
    train_max_analyze['AUC'].append(current_fold['ROC_AUC_SCORE']['Macro'])

train_max_analyze_df = pd.DataFrame(train_max_analyze)
train_max_analyze_df

,Fold,ACC,AP,ASens,ASpec,AF1,AUC
0,fold_0,0.777778,0.767857,0.708333,0.916667,0.723077,0.708333
1,fold_1,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
2,fold_2,0.666667,0.625000,0.625000,0.750000,0.625000,0.625000
3,fold_3,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
4,fold_4,0.777778,0.875000,0.666667,1.000000,0.678571,0.666667
5,fold_5,0.777778,0.762500,0.791667,0.750000,0.766234,0.791667
6,fold_6,0.944444,0.928571,0.958333,0.916667,0.939799,0.958333
7,fold_7,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
8,fold_8,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
9,fold_9,0.769231,0.616667,0.659091,0.818182,0.628571,0.659091


# N_times_K_folds

In [24]:
# Setup

train_max_param = train_max['param']
n_times = 100

nt_kf_output_dict = {
    'Train' : [], # <index>_<fold>
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for n in range(n_times):
    print("Training {}".format(n))
    fold_index = 0
    for train_index, test_index in zip(kmean_train_paths, kmean_test_paths):
        train_df = pd.read_csv(train_index)
        test_df = pd.read_csv(test_index)      
    
        print("\tFold: {}".format(fold_index))
        print("\tTRAIN:", train_index, "\n\tTEST:", test_index)

        # folding data
        x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
        x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
        
        # Training
        print("\t\tTraining : {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
        model = DecisionTreeClassifier(criterion = train_max_param['criterion'], 
                                      splitter = train_max_param['splitter'], 
                                      max_features = train_max_param['max_features'], 
                                      class_weight = train_max_param['class_weight'])
        model.fit(x_train, y_train)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))

        # Testing
        print("\t\tValidation: {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
        y_pred = model.predict(x_test)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
        # Evaluation
        cm = confusion_matrix(y_test, y_pred)
        base_result = utils(cm, y_test, y_pred).get_value()
        
        # Result Storing
        nt_kf_output_dict['Train'].append('{0}_{1}'.format(n, fold_index))
        cls = base_result['Classification Report']
        nt_kf_output_dict['ACC'].append(cls['accuracy'])
        nt_kf_output_dict['AP'].append(cls['macro avg']['precision'])
        nt_kf_output_dict['ASens'].append(cls['macro avg']['recall'])
        nt_kf_output_dict['ASpec'].append(base_result['Confusion Matrix']['specificity'])
        nt_kf_output_dict['AF1'].append(cls['macro avg']['f1-score'])
        nt_kf_output_dict['AUC'].append(base_result['ROC_AUC_SCORE']['Macro'])
        
        # Fold index increment
        fold_index += 1

Training 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 0 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 1 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index2.csv 
	TEST:

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index8.csv
		Training : 8 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 8 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index9.csv
		Training : 9 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 9 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
Training 2
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv 
	TEST:

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index6.csv
		Training : 6 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 6 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index7.csv
		Training : 7 -- Start: 11/29/2022, 09:26:23 --- End: 11/29/2022, 09:26:23
		Validation: 7 -- Start: 11/29/2022, 09:26:23---End: 11/29/2022, 09:26:23
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index8.csv 
	TEST: D:\Github\

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/29/2022, 09:26:47
		Validation: 8 -- Start: 11/29/2022, 09:26:47---End: 11/29/2022, 09:26:47
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index9.csv
		Training : 9 -- Start: 11/29/2022, 09:26:47 --- End: 11/29/2022, 09:26:47
		Validation: 9 -- Start: 11/29/2022, 09:26:47---End: 11/29/2022, 09:26:47
Training 98
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:26:47 --- End: 11/29/2022, 09:26:47
		Validation: 0 -- Start: 11/29/2022, 09:26:47---End: 11/29/2022, 09:26:47
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Cla

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index7.csv
		Training : 7 -- Start: 11/29/2022, 09:26:47 --- End: 11/29/2022, 09:26:47
		Validation: 7 -- Start: 11/29/2022, 09:26:47---End: 11/29/2022, 09:26:47
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_test_0_4_3_True_Falsehisto_128_cv_index8.csv
		Training : 8 -- Start: 11/29/2022, 09:26:47 --- End: 11/29/2022, 09:26:47
		Validation: 8 -- Start: 11/29/2022, 09:26:47---End: 11/29/2022, 09:26:47
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_128\surf_train_0_4_3_True_Falsehisto_128_cv_index9.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_2908\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


In [25]:
nt_kf_output_df = pd.DataFrame(nt_kf_output_dict)
nt_kf_output_df.head(10)

,Train,ACC,AP,ASens,ASpec,AF1,AUC
0,0_0,0.666667,0.625000,0.625000,0.750000,0.625000,0.625000
1,0_1,0.666667,0.607143,0.583333,0.833333,0.584615,0.583333
2,0_2,0.777778,0.762500,0.791667,0.750000,0.766234,0.791667
3,0_3,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
4,0_4,0.944444,0.928571,0.958333,0.916667,0.939799,0.958333
5,0_5,0.777778,0.750000,0.750000,0.833333,0.750000,0.750000
6,0_6,0.555556,0.537500,0.541667,0.583333,0.532468,0.541667
7,0_7,0.888889,0.875000,0.916667,0.833333,0.883117,0.916667
8,0_8,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
9,0_9,0.692308,0.569444,0.613636,0.727273,0.566667,0.613636


In [26]:
nt_kf_output_df.mean()

Train         inf
ACC      0.769692
AP       0.740508
ASens    0.729970
ASpec    0.833106
AF1      0.724618
AUC      0.729970
dtype: float64

In [27]:
nt_kf_output_df.std(numeric_only=True)

ACC      0.103988
AP       0.130000
ASens    0.125796
ASpec    0.108192
AF1      0.124146
AUC      0.125796
dtype: float64

In [28]:
acc_values = nt_kf_output_df['ACC'].values.tolist()
ap_values = nt_kf_output_df['AP'].values.tolist()
asens_values = nt_kf_output_df['ASens'].values.tolist()
aspec_values = nt_kf_output_df['ASpec'].values.tolist()
af1_values = nt_kf_output_df['AF1'].values.tolist()
auc_values = nt_kf_output_df['AUC'].values.tolist()
print(len(acc_values), len(ap_values), len(asens_values), len(aspec_values), len(af1_values), len(auc_values))

1000 1000 1000 1000 1000 1000


In [29]:
import scipy.stats as st

def CI(data, confidence_level, dis_type = 't'):
    if dis_type == 't':
        return st.t.interval(confidence=confidence_level, df=len(data)-1, 
                             loc=np.mean(data), scale=st.sem(data))
    elif dis_type == 'g':
        return st.norm.interval(confidence=confidence_level, loc=np.mean(data), scale=st.sem(data))

def CI_calculator(confidence_level, dis_type = 't'):
    for x in nt_kf_output_df.columns[1:]:
        base = nt_kf_output_df[x].values.tolist()
        print("CI of {}".format(x), CI(base, confidence_level, dis_type))

In [30]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 't')

CI of ACC (0.7632393530017113, 0.7761452623829043)
CI of AP (0.7324411327430808, 0.7485753112704222)
CI of ASens (0.7221634504838843, 0.7377759434555098)
CI of ASpec (0.8263922356031587, 0.8398198856089626)
CI of AF1 (0.7169139090254691, 0.7323215679734748)
CI of AUC (0.7221634504838843, 0.7377759434555098)


In [31]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7632471710848732, 0.7761374442997424)
CI of AP (0.732450906432462, 0.7485655375810409)
CI of ASens (0.7221729081489593, 0.7377664857904348)
CI of ASpec (0.8264003697439861, 0.8398117514681351)
CI of AF1 (0.7169232426071198, 0.7323122343918241)
CI of AUC (0.7221729081489593, 0.7377664857904348)


In [32]:
cl = 0.99

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7612219623850691, 0.7781626529995465)
CI of AP (0.7299191143864571, 0.7510973296270459)
CI of ASens (0.7197229795420033, 0.7402164143973908)
CI of ASpec (0.8242932889636649, 0.8419188322484563)
CI of AF1 (0.7145054567678215, 0.7347300202311224)
CI of AUC (0.7197229795420033, 0.7402164143973908)
